In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp core

# rtime

> "Basic time functions to ensure time is handled consistently across projects."


In [ ]:
# hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
# export
import datetime
import time
from typing import cast, NewType

In [ ]:
# export
UtcTime = NewType("UtcTime", datetime.datetime)
LocalTime = NewType("LocalTime", datetime.datetime)
PosixTime = NewType("PosixTime", int)
MILLIS_IN_A_SECOND = 1000.0
MICROS_IN_A_MILLI = 1000.0

In [ ]:
# exporti
def _seconds_to_posix(seconds: float) -> PosixTime:
    """Convert the seconds to a PosixTime."""
    return cast(PosixTime, int(MILLIS_IN_A_SECOND * seconds))

In [ ]:
# export
def seconds_to_millis(seconds: float) -> int:
    """Convert the seconds to milliseconds.
    
    Valuse are rounded to the nearest millisecond.
    """
    return int(MILLIS_IN_A_SECOND * seconds + 0.5)

In [ ]:
test_eq(seconds_to_millis(2.2), 2200)
test_eq(seconds_to_millis(0.1), 100)
test_eq(seconds_to_millis(0.02), 20)
test_eq(seconds_to_millis(0.003), 3)
test_eq(seconds_to_millis(0.0004), 0)
test_eq(seconds_to_millis(0.0005), 1)

In [ ]:
# export
def timestamp() -> PosixTime:
    """Return the current Posix time in milliseconds."""
    return _seconds_to_posix(seconds=time.time())

Unlike python's builtin timetsamp function time.time `timestamp` returns the current time in milliseconds since the epoch rather than seconds.

In [ ]:
test_close(timestamp() / time.time(), 1000.0)

In [ ]:
# export
def to_utc(timestamp: PosixTime) -> UtcTime:
    """Convert a timestamp to utc time."""
    seconds = timestamp / MILLIS_IN_A_SECOND
    dt = datetime.datetime.fromtimestamp(seconds, datetime.timezone.utc)
    return cast(UtcTime, dt)

In [ ]:
now = timestamp()
utc = to_utc(timestamp=now)
# Verify that the object has the utc timezone.
test_eq(utc.tzinfo, datetime.timezone.utc)
# Verify we still have the same time when converting back to a timestamp.
# utc.timestamp produces a time in seconds where as now is in milliseconds.
test_eq(now / utc.timestamp(), 1000.0)

In [ ]:
# export
def to_local(timestamp: PosixTime) -> LocalTime:
    """Convert the timestamp to a time in the local timezone."""
    return cast(LocalTime, to_utc(timestamp=timestamp).astimezone())

In [ ]:
start = datetime.datetime.now().astimezone()
timestamp = _seconds_to_posix(seconds=start.timestamp())
local_from_ts = to_local(timestamp=timestamp)
# Assert the local time created from the timestamp has the local timezone.
test_eq(local_from_ts.tzinfo, start.tzinfo)
diff = (start - local_from_ts).total_seconds()
# Our timestamps only have millisecond precision so the conversion to a timestamp
# and back to a local time may not exactly equally the starting time.
test_close(diff, 0, eps=1e-03)
# Since we truncate timestamps to the neareast millisecond the conversion back to
# a local time should never be greater than the start.
test_eq(diff >= 0, True)

In [ ]:
# hide
from nbdev.export import notebook2script

notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
